# Meshing from Neper geometries

Neper is a free / open source software package for polycrystal generation and meshing, see https://neper.info

To run this notebook you first have to install neper.

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import subprocess, random

n = 100
subprocess.run(["neper", "-T", "-n", str(n), "-format", "tess"])

In [ ]:
verts = {}
edges = {}
faces = {} 
solids = []

f = open("n" + str(n) + "-id1.tess")

def randomcol():
    r = random.uniform(0, 1)
    g = random.uniform(0, 1)
    b = random.uniform(0, 1)
    return (r,g,b)


while True:
    line = f.readline()
    if not line: break

    if line.split()[0] == "**vertex":
        num = int(f.readline())
        print ("found", num, "vertices")

        for i in range(num):
            line = f.readline()            
            nr,x,y,z,hhh = line.split()
            verts[int(nr)] = Vertex(Pnt(float(x), float(y), float(z)))

    if line.split()[0] == "**edge":
        num = int(f.readline())
        print ("found", num, "edges")

        for i in range(num):
            line = f.readline()            
            nr,i1,i2,hhh = line.split()
            edge = Edge(verts[int(i1)], verts[int(i2)])
            edges[int(nr)] = edge
            edges[-int(nr)] = edge.Reversed()
            
    if line.split()[0] == "**face":
        num = int(f.readline())
        print ("found", num, "faces")

        for i in range(num):
            l1 = f.readline()            
            l2 = f.readline()            
            l3 = f.readline()            
            l4 = f.readline()
            nr,*_ = l1.split()
            nume,*fedges = l2.split()
            
            face = Face(Wire( [edges[int(enr)]  for enr in fedges] ))
            faces[int(nr)] = face
            faces[-int(nr)] = face.Reversed()

            
    if line.split()[0] == "**polyhedron":
        num = int(f.readline())
        print ("found", num, "polyhedra")

        for i in range(num):
            nr,nrf,*polyfaces = f.readline().split()
            solids.append (Solid(Glue( [faces[int(fnr)] for fnr in polyfaces ] )))
            solids[-1].faces.col = randomcol()                        
            
shape = Glue(solids)
geo = OCCGeometry(shape)

Draw (shape);

In [ ]:
mp = netgen.meshing.MeshingParameters(segmentsperedge=0.1)
with TaskManager(pajetrace=10**9):
    mesh = geo.GenerateMesh(maxh=0.05, mp=mp)

print ("ne =", mesh.ne)

In [ ]:
Draw (mesh); 